# Problem Set 4

In [1]:
using InstantiateFromURL
# optionally add arguments to force installation: instantiate = true, precompile = true
github_project("QuantEcon/quantecon-notebooks-julia", version = "0.8.0")

Activated /Users/taisei/Project.toml
Info Project name is quantecon-notebooks-julia, version is 0.8.0


In [2]:
using LinearAlgebra, Statistics
using Distributions, Plots, QuadGK, Polynomials, Interpolations
gr(fmt = :png);

┌ Info: Precompiling Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1260
┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1260
┌ Warning: Module JSON with build ID 33541892065587 is missing from the cache.
│ This may mean JSON [682c06a0-de6a-54ab-a142-c8b1cf79cde6] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1016
┌ Info: Skipping precompilation since __precompile__(false). Importing Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80].
└ @ Base loading.jl:1033
┌ Info: Precompiling GR [28b8d3ca-fb5f-59d9-8090-bfdbd6d07a71]
└ @ Base loading.jl:1260
┌ Warning: Module JSON with build ID 33541892065587 is missing from the cache.
│ This may mean JSON [682c06a0-de6a-54ab-a142-c8b1cf79cde6] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1016
┌ Info: Skipping precompilation since __precompile__(false). Importing GR [28b8d3ca-fb5f-59d9-8090-bfdbd6d07a7

## Generic Programming

### Exercise 1 (a)

* Benchmark

In [9]:
using LinearAlgebra
function trap_weights(x)
    return step(x) * [0.5; ones(length(x) - 2); 0.5]
end
x = range(0.0, 1.0, length = 100)
ω = trap_weights(x)
f(x) = x^2
dot(f.(x), ω)

0.3333503384008434

* Use iteration

In [35]:
struct UniformTrapezoidal
    count::Int
    Δ::Float64
end
Base.iterate(S::UniformTrapezoidal,state=1)= state > S.count ? nothing : 
((state==1 || state == S.count) ? (S.Δ*0.5,state+1) : (S.Δ,state+1))


0.3333503384008434

In [ ]:
x = range(0.0, 1.0, length = 100)
ω = UniformTrapezoidal(length(x),step(x))
f(x) = x^2
dot(f.(x),ω)

In [117]:
#Version 2.0 try not to use dot product

function trapz_in(f, nodes::AbstractRange)
    M = length(nodes)
    Δ = step(nodes)
    total = zero(eltype(nodes))
    for (i,x) in enumerate(nodes)
        weight = ((i == 1) || (i == M)) ? Δ/2 : Δ
        total += weight * f(x)
    end
    return total
end
x = 0.0:0.01:1.0
f(x) = x^2
trapz_in(f,x)

0.33335000000000015

### Exercise 1(b)

In [43]:
function Base.getindex(S::UniformTrapezoidal,i::Int)
    1<=i<=S.count || throw(BoundsError(S,i))
    if i==1||i==S.count
        return S.Δ*0.5
    else
        return S.Δ
    end
end

In [46]:
Base.length(S::UniformTrapezoidal)=S.count

In [41]:
function Base.getindex(S::Squares,i::Int)
    1<=i<=S.count || throw(BoundsError(S,i))
    if i==1
        return 
end
Squares(100)[23]

529

In [37]:
ω = UniformTrapezoidal(length(x),step(x))

UniformTrapezoidal(100, 0.010101010101010102)

In [44]:
ω[2]

0.010101010101010102

In [47]:
length(ω)

100

### Exercise 2 

In [35]:
struct UniformTrapezoidal
    count::Int
    Δ::Float64
end
Base.iterate(S::UniformTrapezoidal,state=1)= state > S.count ? nothing : 
((state==1 || state == S.count) ? (S.Δ*0.5,state+1) : (S.Δ,state+1))


0.3333503384008434

In [66]:
struct NonUniformTrapezoidal
    count::Int
    Δ::Array{Float64,1}
end
Base.iterate(S::NonUniformTrapezoidal,state=1)= state > S.count ? nothing : 
(state == 1 ? (0.5*(S.Δ[2]-S.Δ[1]) , state+1) : (state == S.count ? (0.5*(S.Δ[end]-S.Δ[end-1]), state+1) :
        (S.Δ[state]-S.Δ[state-1] , state+1)))

In [67]:
x = range(0.0, 1.0, length = 100)
x2 = x.^2
ω = NonUniformTrapezoidal(length(x2),x2)
f(x) = x^2
dot(f.(x),ω)

0.4966838400671685

Too large?

In [87]:
x = 0.0:0.01:1.0
x_array=Array(x)
ω = NonUniformTrapezoidal(length(x_array),x_array)
f(x) = x^2
dot(f.(x_array),ω)

0.33335

The function seems to work well.

* A [Notebook]("https://nbviewer.jupyter.org/github/ubcecon/ECON622_2019/blob/master/notebooks/quadrature.ipynb") from ECON622_2019 

Attempt to write a multi-dispatch function.

In [88]:
function trapz_weights(nodes::AbstractArray)  # Note multiple dispatch
    println("trapz for AbstractArray:")
    M = length(nodes)
    Δ = diff(nodes)
    prepend!(Δ, NaN) # To keep the indexing straight. Now, Δ[2] = Δ_2 = z_2 - z_1. And NaN will throw an error if we try to use it.
    interiorWeights = [(Δ[i] + Δ[i+1])/2 for i = 2:M-1]
    return [Δ[2]/2; interiorWeights; Δ[M]/2]
end

trapz_weights (generic function with 1 method)

In [90]:
# Specialization.  New method added to `trapz_weights`
function trapz_weights(nodes::AbstractRange)
    println("trapz for AbstractRange:")
    M = length(nodes)
    Δ = step(nodes)
    return [Δ/2; Δ*ones(M-2); Δ/2]
end

trapz_weights (generic function with 2 methods)

In [89]:
w = trapz_weights(x_array)  # call with array
@show w ⋅ f.(x_array);

trapz for AbstractArray:
w ⋅ f.(x_array) = 0.33335


In [91]:
w = trapz_weights(x) # call with range
@show w ⋅ f.(x);

trapz for AbstractRange:
w ⋅ f.(x) = 0.33335000000000004


## Git and Github